**Prerequisites to running this notebook:**
1. Install the following in your dev environment:<br>
    a. google-cloud-bigquery: pip.exe install google-cloud-bigquery<br>
    b. db-types: pip install db-dtypes<br>
2. Install gcloud CLI <br>
    a. Install directions (with download link): https://cloud.google.com/sdk/docs/install<br>
    > i. pay attention to where it installs to add this to your PATH environmental variables<br>
    > ii. It says to leave all the shortcut, open terminal options checked. I received errors when it ran "gcloud info --run-diagnostics" and I ignored them until I updated the PATH envrionmental variables<br>
    
    b. Add this to your PATH environmental variables (for me this was C:\Users\vt_be\AppData\Local\Google\Cloud SDK\google-cloud-sdk)<br>
    c. reboot!<br>
    d. open git bash, switch to dev environment<br>
    > i. "gcloud info --run-diagnostics" now ran without issue<br>
    ii. add authentication (this opens browser to connect your google account):  gcloud auth application-default login<br>
    
    e. I also needed to set up a Big Query Project: mostly followed https://cloud.google.com/bigquery/docs/sandbox<br>
    > i. I didn't see the stuff mentioned in #3 but it otherwise worked as written<br>
    > ii. Note that when you create the project, an id is generated that is project name - #### (for me BootCamp-Weather:  bootcamp-weather-400118<br>
    
    f. Add the project to default - back to gitbash: gcloud auth application-default set-quota-project <project-id><br>
3. Create a config.py file in the folder that contains this notebook. 
4. Add the following variable to config.py: google_project = "your-project-id"
    

**Credit:**
* Big Query calls adapted from https://www.kaggle.com/code/crained/noaa-dataset-with-google-bigquery
* SQL calls adapted from GitHub BigQuery documentation: https://github.com/googleapis/python-bigquery

**Data Info:**
https://data.noaa.gov/dataset/dataset/global-surface-summary-of-the-day-gsod

In [1]:
# Since this is unique to user, I added config.py to the gitignore. 
# You must create your own config.py file with project name as stated in the Prerequisites
from config import google_project
# bigquery and pandas work well together for dataframes!
import pandas as pd
import os
# Follow the prerequisite instructions to get bigquery going
from google.cloud import bigquery
# Create a "Client" object reference a google project for which your system has been authenticated
client = bigquery.Client(google_project)

## Query for the Stations static info ##
usaf for connecting tables, <br>
spatial data required for heatmap, <br>
name and state for usability <br>

In [2]:
# Station IDs (usaf) remained consistent to the physical station over time, but names changed resulting in multiple entries with same usaf. 
# This query will sort by begin date descending so we can use the pandas remove duplicates function to keep the latest name only.

# Define the SQl suery
QUERY = (
    'SELECT usaf, name, state, lat, lon, elev '
    'FROM `bigquery-public-data.noaa_gsod.stations` '
    'WHERE country = "US" AND state <> "None" '
    'ORDER BY begin DESC'
    )
# API request
stations_result = client.query(QUERY)  
# Waits for query to finish
stations_data = stations_result.result()  

# Put the query into a dataframe
stations_data_df = stations_data.to_dataframe()

# Remove duplicate stations by usaf
stations_data_df = stations_data_df.drop_duplicates("usaf")

# # and export if desired (not required for final implementation)
# stations_data_df.to_json("./data/Stations.json", orient="records")
# stations_data_df.to_csv("./data/Stations.csv")

print(f' There are {len(stations_data_df)} rows')
stations_data_df.head()

 There are 3790 rows


,usaf,name,state,lat,lon,elev
0,720511,BRITTON MUNI,SD,45.815,-97.743,+0401.7
1,723062,PINEY ISLAND,NC,35.020,-76.460,+0005.2
2,A00030,CONNELLSVILLE AIRPORT,PA,39.959,-79.657,+0386.2
3,720844,SPANISH PEAKS,CO,37.697,-104.785,+1844.0
4,724916,MARINA MUNI,CA,36.682,-121.762,+0040.8


## Query for the temperature data ##
### Query for temperature stats (similar filtering) by station ###
SchemaField('min', 'FLOAT', 'NULLABLE', None, 'Minimum temperature reported during the day in Fahrenheit to tenths--time of min temp report varies by country and region, so this will sometimes not be the min for the calendar day. Missing = 9999.9', (), None),<br>
SchemaField('temp', 'FLOAT', 'NULLABLE', None, 'Mean temperature for the day in degrees Fahrenheit to tenths. Missing = 9999.9', (), None),<br>
SchemaField('max', 'FLOAT', 'NULLABLE', None, 'Maximum temperature reported during the day in Fahrenheit to tenths--time of max temp report varies by country and region, so this will sometimes not be the max for the calendar day. Missing = 9999.9', (), None),

In [3]:
# Define the query to get absoulte minimum temperature, average mean temperature, absolute maximum temperature 
# for the year by station (usaf)
aggregate_query = (
    'SELECT s.usaf, '
    'MIN(g.min) AS min_temp, '
    'AVG(g.temp) AS mean_temp, '
    'MAX(g.max) AS max_temp, '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    # The line below removes the 'not a reading' so we can run stats on those columns
    'AND g.min <> 9999.9 AND g.max <> 9999.9 '
    'GROUP BY s.usaf '
    )

# API request
station_temp_result = client.query(aggregate_query)  
# Waits for query to finish
station_temp_data = station_temp_result.result()  

# Put the query into a dataframe
station_temp_df = station_temp_result.to_dataframe()

# # and export if desired (not required for final implementation)
# state_temp_station.to_json("./data/Station_temp_sample.json", orient="records")
# state_temp_station.to_csv("./data/Station_temp_sample.csv")

print(f' There are {len(station_temp_df)} rows')
station_temp_df.head()

 There are 2522 rows


,usaf,min_temp,mean_temp,max_temp
0,701043,-27.9,21.075649,54.0
1,702040,-5.1,30.590909,55.9
2,702490,-27.4,32.636119,77.0
3,702606,-7.6,24.352830,55.4
4,703406,-22.0,42.212132,82.4


### Query for total snow in a year by station ###
SchemaField('sndp', 'FLOAT', 'NULLABLE', None, "Snow depth in inches to tenths--last report for the day if reported more thanonce. Missing = 999.9 Note: Most stations do not report '0' ondays with no snow on the ground--therefore, '999.9' will often appear on these days", (), None)

In [4]:
# Perform a query for the sum of snow depth for each station over the year
aggregate_query = (
    'SELECT s.usaf, '
    'SUM(g.sndp) AS total_snow '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    # The line below removes the 'not a reading' so we can run stats on those columns
    'AND g.sndp <> 999.9 '
    'GROUP BY s.usaf '
    'ORDER BY total_snow DESC')
station_snow_result = client.query(aggregate_query)  # API request
station_snow_data = station_snow_result.result()  # Waits for query to finish

# Put the query into a dataframe
station_snow_df = station_snow_result.to_dataframe()

print(f' There are {len(station_snow_df)} rows')
station_snow_df.head()

 There are 281 rows


,usaf,total_snow
0,702606,14721.2
1,702650,8622.6
2,702490,7887.4
3,702615,6871.8
4,701740,5199.6


### Query for total precipitation in a year ###
SchemaField('prcp', 'FLOAT', 'NULLABLE', None, "Total precipitation (rain and/or melted snow) reported during the day in inches and hundredths; will usually not end with the midnight observation--i.e., may include latter part of previous day.  .00 indicates no measurable precipitation (includes a trace). Missing = 99.99 Note: Many stations do not report '0' on days with no precipitation--therefore, '99.99' will often appear on these days. 

**TODO:** Understand this to see if need to include a filter around teh flag
Also, for example, a station may only report a 6-hour amount for the period during which rain fell. See Flag field for source of data", (), None),


 SchemaField('flag_prcp', 'STRING', 'NULLABLE', None, "A = 1 report of 6-hour precipitation amount B = Summation of 2 reports of 6-hour precipitation amount C = Summation of 3 reports of 6-hour precipitation amount D = Summation of 4 reports of 6-hour precipitation amount E = 1 report of 12-hour precipitation amount F = Summation of 2 reports of 12-hour precipitation amount G = 1 report of 24-hour precipitation amount H = Station reported '0' as the amount for the day (eg, from 6-hour reports), but also reported at least one occurrence of precipitation in hourly observations--this could indicate a trace occurred, but should be considered as incomplete data for the day. I = Station did not report any precip data for the day and did not report any occurrences of precipitation in its hourly observations--it's still possible that precip occurred but was not reported", (), None),

In [5]:
# Perform a query for the sum of precipitation for each station over the year
aggregate_query = (
    'SELECT s.usaf, '
    'SUM(g.prcp) AS total_precipitation '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    # The line below removes the 'not a reading' so we can run stats on those columns
    'AND g.prcp <> 99.9 '
    'GROUP BY s.usaf')
station_prcp_result = client.query(aggregate_query)  # API request
station_prcp_data = station_prcp_result.result()  # Waits for query to finish

# Put the last query into a dataframe
station_prcp_df = station_prcp_result.to_dataframe()

print(f' There are {len(station_prcp_df)} rows')
station_prcp_df.head()

 There are 2523 rows


,usaf,total_precipitation
0,700860,12400.22
1,702040,8000.06
2,702223,1229.90
3,702490,10613.66
4,702606,8199.18


### Query for count of days with tornadoes in a year ###
SchemaField('tornado_funnel_cloud', 'STRING', 'NULLABLE', None, 'Indicators (1 = yes, 0 = no/not reported) for the occurrence during the day', (), None)

In [6]:
# Perform a query that pulls only the stations reporting a tornado to allow counting how many days each station had a tornado 
# Include month and day so can remove duplicate "yes" on the same day
QUERY = (
    'SELECT s.usaf, g.mo, g.da, '
    'g.tornado_funnel_cloud '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    'AND tornado_funnel_cloud = "1" '
    )
station_tornado_result = client.query(QUERY)  # API request
station_tornado_data = station_tornado_result.result()  # Waits for query to finish

# Put the query into a dataframe
station_tornado_df = station_tornado_result.to_dataframe()

# Remove duplicate station day reports
station_tornado_nodup = station_tornado_df.drop_duplicates()
# Drop month and day columns
station_tornado_nodup = station_tornado_nodup[["usaf", "tornado_funnel_cloud"]]
# Count how many in the year for each station and store in a new dataframe
station_tornado_count = station_tornado_nodup.groupby("usaf").count()

print(f' There are {len(station_tornado_count)} rows')
station_tornado_count.head()

 There are 33 rows


,tornado_funnel_cloud
usaf,
720381,1
722010,4
722015,8
722020,2
722030,1


### Query for count of days with hail in a year ###
SchemaField('hail', 'STRING', 'NULLABLE', None, 'Indicators (1 = yes, 0 = no/not reported) for the occurrence during the day', (), None)

In [7]:
# Perform a query that pulls only the stations reporting hail to allow counting how many days each station had a tornado 
# Include month and day so can remove duplicate "yes" on the same day
QUERY = (
    'SELECT s.usaf, g.mo, g.da, '
    'g.hail '
    'FROM `bigquery-public-data.noaa_gsod.gsod2022` AS g '
    'INNER JOIN `bigquery-public-data.noaa_gsod.stations` AS s ON g.stn = s.usaf '
    'WHERE s.country = "US" AND s.state <> "None" '
    'AND g.hail = "1" '
    )
station_hail_result = client.query(QUERY)  # API request
station_hail_data = station_hail_result.result()  # Waits for query to finish

# Put the query into a dataframe
station_hail_df = station_hail_result.to_dataframe()

# Remove duplicate station day reports
station_hail_nodup = station_hail_df.drop_duplicates()
# Drop month and day
station_hail_nodup = station_hail_nodup[["usaf", "hail"]]
# Count how many in the year for each station and store in a new dataframe
station_hail_count = station_hail_nodup.groupby("usaf").count()

print(f' There are {len(station_hail_count)} rows')
station_hail_count.head()

 There are 113 rows


,hail
usaf,
702650,2
702730,1
702910,1
704140,12
720619,1


## Merge all the data frames together ##

In [8]:
# Use an inner here to create a basis for all possible without adding NaN rows
station_temp_merge = pd.merge(stations_data_df, station_temp_df, how ="inner", on = "usaf")

# Use outer here to include everything as the snow, precipitation, tornado, and hail dataframes
# does not include stations that did not register readings but have valid temperature readings
st_temp_snow_merge = pd.merge(station_temp_merge, station_snow_df, how ="outer", on = "usaf")

st_temp_snow_prcp = pd.merge(st_temp_snow_merge, station_prcp_df, how ="outer", on = "usaf")

station_stats_torn = pd.merge(st_temp_snow_prcp, station_tornado_count, how ="outer", on = "usaf")

station_all = pd.merge(station_stats_torn, station_hail_count, how ="outer", on = "usaf")

# Looking at the csv file, the total snow = 0 brought in a 0 where temperatures and hail are NaN
# Removing those rows
station_all = station_all.dropna(subset=["min_temp", "mean_temp", "max_temp"])

# Convert the remaining NaN to zero for reporting and future math
station_all = station_all.fillna(0)

# Sort by states and name for easy dropdown menu population
station_all = station_all.sort_values(by=["state", "name"], ascending = [True, True])

# # and export if desired
# station_all.to_json("./data/stations_all.json", orient="records")
# station_all.to_csv("./data/stations_all.csv")
# station_all.to_json("./data/stations_all.js", orient="records")

print(f' There are {len(station_all)} rows')
station_all.head()

 There are 2522 rows


,usaf,name,state,lat,lon,elev,min_temp,mean_temp,max_temp,total_snow,total_precipitation,tornado_funnel_cloud,hail
2000,704540,ADAK (NAS),AK,51.883,-176.650,+0005.0,12.9,41.957418,69.1,0.0,2.924200e+02,0.0,0.0
682,997380,ADAK ISLAND,AK,51.870,-176.630,+0007.0,18.7,42.015254,64.9,0.0,0.000000e+00,0.0,0.0
738,703926,AKHIOK,AK,56.933,-154.183,+0013.0,8.1,37.401887,71.1,0.0,3.092500e+02,0.0,0.0
547,702686,AKIAK,AK,60.903,-161.231,+0009.1,-20.0,34.427976,75.9,0.0,1.220168e+04,0.0,0.0
13,999999,ALEKNAGIK 1 NNE,AK,59.284,-158.615,+0024.4,-57.1,52.816638,122.4,0.0,4.902942e+08,0.0,0.0


### Add temporary severity rating ###

In [9]:
station_all["severity_rating"] = station_all["tornado_funnel_cloud"]*3 + station_all["hail"]

# and export
station_all.to_json("../data/stations_all.json", orient="records")
station_all.to_csv("../data/stations_all.csv")
station_all.to_json("../data/stations_all.js", orient="records")

station_all.head()

,usaf,name,state,lat,lon,elev,min_temp,mean_temp,max_temp,total_snow,total_precipitation,tornado_funnel_cloud,hail,severity_rating
2000,704540,ADAK (NAS),AK,51.883,-176.650,+0005.0,12.9,41.957418,69.1,0.0,2.924200e+02,0.0,0.0,0.0
682,997380,ADAK ISLAND,AK,51.870,-176.630,+0007.0,18.7,42.015254,64.9,0.0,0.000000e+00,0.0,0.0,0.0
738,703926,AKHIOK,AK,56.933,-154.183,+0013.0,8.1,37.401887,71.1,0.0,3.092500e+02,0.0,0.0,0.0
547,702686,AKIAK,AK,60.903,-161.231,+0009.1,-20.0,34.427976,75.9,0.0,1.220168e+04,0.0,0.0,0.0
13,999999,ALEKNAGIK 1 NNE,AK,59.284,-158.615,+0024.4,-57.1,52.816638,122.4,0.0,4.902942e+08,0.0,0.0,0.0


## Update js export to work with our JavaScript code ##
For the use with our javascript, we want "let stations_all = " to precede the data

In [10]:
with open("../data/stations_all.js", 'r+') as file:
    # read the contents into a variable
    content = file.read()
    # set cursor to start of file
    file.seek(0, 0)
    # write the desired starter text and rewrite the original content
    file.write("let stations_all = " + content)
    file.close()